<a href="https://colab.research.google.com/github/lpdata/fraude_bilhetagem/blob/main/notebooks/02_tratamento_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamento de Features

## 0. Contexto e objetivo

Este notebook tem como objetivo consolidar o tratamento dos dados e a engenharia de features a partir do conjunto explorado na etapa anterior. Após a compreensão da estrutura dos dados, dos padrões comportamentais e dos principais sinais associados à ocorrência de fraude, esta etapa é responsável por transformar essas evidências exploratórias em atributos estruturados, reprodutíveis e adequados para a modelagem preditiva.

As atividades realizadas aqui incluem a padronização de tipos e schema, o tratamento de inconsistências e valores inválidos, bem como a criação de features temporais, comportamentais e operacionais fundamentadas nas análises exploratórias previamente conduzidas. Todas as transformações são aplicadas de forma determinística e sem utilização da variável alvo como insumo, garantindo a mitigação de vazamento de informação.

Ao final deste notebook, é gerado um dataset processado e documentado, pronto para ser utilizado na etapa de modelagem, preservando rastreabilidade, clareza metodológica e alinhamento com as boas práticas de projetos de ciência de dados aplicados à detecção de fraude.

## 1. Vinculação com Github

In [1]:
!git clone https://github.com/lpdata/fraude_bilhetagem

Cloning into 'fraude_bilhetagem'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 95 (delta 40), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 3.21 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [2]:
%cd fraude_bilhetagem

/content/fraude_bilhetagem


In [3]:
!ls

data  notebooks  README.md  requirements.txt  src


In [4]:
!ls data/raw

dados.csv


In [5]:
import os
print("Diretório atual:", os.getcwd()) #checando se estou no diretório correto

Diretório atual: /content/fraude_bilhetagem


## 2. Imports e configurações

In [6]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', '{:.4f}'.format)


In [9]:
RANDOM_STATE = 42 #Seed para reprodutibilidade
np.random.seed(RANDOM_STATE)


In [10]:
# Definição dos diretórios do projeto

BASE_DIR = Path.cwd()

DATA_DIR = BASE_DIR / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUTS_DIR = BASE_DIR / 'outputs'

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)


In [11]:
print("Diretório base:", BASE_DIR)
print("Arquivos em data/raw:", list(RAW_DIR.iterdir()))


Diretório base: /content/fraude_bilhetagem
Arquivos em data/raw: [PosixPath('/content/fraude_bilhetagem/data/raw/.gitkeep'), PosixPath('/content/fraude_bilhetagem/data/raw/dados.csv')]


In [16]:
def checkpoint(msg):
    print(f"✔ {msg}")

checkpoint("Etapa 2 concluída: imports, configurações e diretórios prontos")

✔ Etapa 2 concluída: imports, configurações e diretórios prontos


<small>***Comentários Letícia:** Nesta etapa, realizei a configuração inicial do ambiente de trabalho, incluindo a importação das bibliotecas necessárias, a definição de parâmetros globais e a validação da estrutura do projeto. Essas configurações garantem reprodutibilidade, organização do código e consistência nas etapas subsequentes de tratamento e engenharia de features.*</small>

## 3. Carregamento dos dados e validação de integridade

### 3.1 Carregar o dataset raw

In [17]:
DATA_PATH = RAW_DIR / "dados.csv"

df = pd.read_csv(DATA_PATH)

checkpoint(f"Dados carregados com sucesso: {DATA_PATH}")
checkpoint(f"Shape do dataset: {df.shape}")
display(df.head())


✔ Dados carregados com sucesso: /content/fraude_bilhetagem/data/raw/dados.csv
✔ Shape do dataset: (30000, 28)


,id_transacao,id_cartao,ts_transacao,dt_emissao_cartao,nu_recargas_acumulado,valor_transacao,tipo_cartao,status_cartao,id_dispositivo,id_motorista,linha_onibus,sentido_viagem,clima_dia,temp_externa,id_loja_ultima_recarga,tipo_pagamento_recarga,latitude,longitude,cidade,feriado,integracao_metro,limite_diario_uso,idade_usuario,sexo_usuario,diff_tempo_segundos,tempo_vida_cartao_dias,target_fraude,bloqueio_automatico_sistema
0,160,18901,2026-01-21 20:23:29,2022-10-17,27,4.5000,Funcionario,Ativo,560,1155,88,Volta,Chuva,24.8087,15,Cartao_Credito,-23.5194,-46.6360,São Paulo,0,0,10,26,N,754063.0000,1192,0,0
1,103,18392,2026-01-29 03:13:36,2023-06-30,142,4.5000,Idoso,Ativo,597,1184,99,Volta,Chuva,26.5893,44,Dinheiro,-23.5968,-46.6113,São Paulo,0,1,10,39,F,1050052.0000,944,0,0
2,22320,17050,2026-01-04 16:05:52,2020-10-29,51,4.5000,Comum,Ativo,555,1118,7,Volta,Chuva,37.0170,13,PIX,-23.5743,-46.6591,São Paulo,0,0,10,45,F,25224.0000,1893,0,0
3,12932,11516,2026-01-15 01:29:07,2023-09-23,34,9.0000,Idoso,Ativo,500,1152,129,Volta,Chuva,29.3219,26,PIX,-23.5572,-46.6581,São Paulo,0,1,10,20,N,NaN,845,0,0
4,21,16396,2026-01-19 00:20:30,2023-01-10,78,4.5000,Vale-Transporte,Ativo,563,1078,111,Ida,Chuva,24.1620,12,Cartao_Credito,-23.5435,-46.6858,São Paulo,0,0,10,45,M,507280.0000,1105,0,0


### 3.2 Visão geral rápida (colunas e tipos atuais)

In [18]:
checkpoint("Visão geral do dataset (dtypes e nulos iniciais)")

display(df.dtypes)
display(df.isna().mean().sort_values(ascending=False).head(15))


✔ Visão geral do dataset (dtypes e nulos iniciais)


,0
id_transacao,int64
id_cartao,int64
ts_transacao,object
dt_emissao_cartao,object
nu_recargas_acumulado,int64
valor_transacao,float64
tipo_cartao,object
status_cartao,object
id_dispositivo,int64
id_motorista,int64


,0
diff_tempo_segundos,0.3161
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000
id_motorista,0.0000


### 3.3 Validação de schema (colunas esperadas)

In [19]:
expected_cols = [
    "id_transacao","id_cartao","ts_transacao","dt_emissao_cartao","nu_recargas_acumulado",
    "valor_transacao","tipo_cartao","status_cartao","id_dispositivo","id_motorista",
    "linha_onibus","sentido_viagem","clima_dia","temp_externa","id_loja_ultima_recarga",
    "tipo_pagamento_recarga","latitude","longitude","cidade","feriado","integracao_metro",
    "limite_diario_uso","idade_usuario","sexo_usuario","bloqueio_auto_sistema","target_fraude"
]

missing_cols = [c for c in expected_cols if c not in df.columns]
extra_cols = [c for c in df.columns if c not in expected_cols]

if missing_cols:
    print("⚠️ Colunas esperadas ausentes:", missing_cols)
else:
    checkpoint("Nenhuma coluna esperada está ausente.")

if extra_cols:
    print("ℹ️ Colunas extras encontradas (não previstas no dicionário):", extra_cols)
else:
    checkpoint("Nenhuma coluna extra encontrada.")


⚠️ Colunas esperadas ausentes: ['bloqueio_auto_sistema']
ℹ️ Colunas extras encontradas (não previstas no dicionário): ['diff_tempo_segundos', 'tempo_vida_cartao_dias', 'bloqueio_automatico_sistema']


### 3.4 Checagem de duplicidade (linhas e chaves)

In [20]:
dup_rows = df.duplicated().sum()
dup_id_transacao = df["id_transacao"].duplicated().sum()

checkpoint(f"Linhas duplicadas completas: {dup_rows}")
checkpoint(f"id_transacao duplicado: {dup_id_transacao}")


✔ Linhas duplicadas completas: 0
✔ id_transacao duplicado: 0


### 3.5 Checagem de valores básicos (sanity checks)

In [21]:
checkpoint("Sanity checks básicos")

# Contagem de valores únicos em chaves
print("Transações únicas:", df["id_transacao"].nunique())
print("Cartões únicos:", df["id_cartao"].nunique())

# Target: distribuição inicial
print("\nDistribuição do target_fraude:")
display(df["target_fraude"].value_counts(dropna=False))


✔ Sanity checks básicos
Transações únicas: 30000
Cartões únicos: 9483

Distribuição do target_fraude:


,count
target_fraude,
0,27065
1,2935


### 3.6 Checkpoint final da etapa 3

In [22]:
checkpoint("Etapa 3 concluída: dados carregados e integridade inicial verificada")


✔ Etapa 3 concluída: dados carregados e integridade inicial verificada


<small>***Comentários Letícia:** Nesta etapa, validei a integridade estrutural do dataset carregado, identificando divergências de schema em relação ao dicionário de dados e a presença de colunas derivadas não previstas no conjunto bruto. Os dados não apresentaram duplicidades nem inconsistências críticas de chave, e a distribuição da variável alvo confirmou um cenário de desbalanceamento moderado. Esses achados orientam diretamente as decisões de padronização e tratamento a serem realizadas nas etapas subsequentes.*</small>

## 4.Padronização de tipos e parsing (schema enforcement)

### 4.1 Parsing de ts_transacao e dt_emissao_cartao

In [23]:
# Parsing de colunas temporais
df['ts_transacao'] = pd.to_datetime(df['ts_transacao'], errors='coerce')
df['dt_emissao_cartao'] = pd.to_datetime(df['dt_emissao_cartao'], errors='coerce')

checkpoint("Parsing de datas concluído: ts_transacao e dt_emissao_cartao convertidas para datetime")


✔ Parsing de datas concluído: ts_transacao e dt_emissao_cartao convertidas para datetime


In [24]:
# Verificação dos tipos após conversão
display(df[['ts_transacao', 'dt_emissao_cartao']].dtypes)

# Checagem de valores nulos gerados no parsing
display(
    df[['ts_transacao', 'dt_emissao_cartao']]
    .isna()
    .mean()
    .mul(100)
    .round(2)
)


,0
ts_transacao,datetime64[ns]
dt_emissao_cartao,datetime64[ns]


,0
ts_transacao,0.0000
dt_emissao_cartao,0.0000


<small>***Comentários Letícia:** A conversão das colunas temporais foi realizada com sucesso, sem geração de valores ausentes, indicando consistência e boa qualidade dos dados de data e hora. Com isso, o dataset está apto para operações temporais e para a criação de features dependentes de tempo nas etapas seguintes.*</small>

### 4.2 Normalização de binárias

In [26]:
# Renomear coluna para alinhar com o dicionário de dados
df = df.rename(columns={'bloqueio_automatico_sistema': 'bloqueio_auto_sistema'})

checkpoint("Coluna 'bloqueio_automatico_sistema' renomeada para 'bloqueio_auto_sistema'")

# Função utilitária para normalização binária
def normalize_binary(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.strip().str.lower()
    mapping = {
        '1': 1, 'true': 1, 'sim': 1, 's': 1,
        '0': 0, 'false': 0, 'nao': 0, 'não': 0, 'n': 0
    }
    return s.map(mapping)

# Aplicar normalização
binary_cols = [
    'feriado',
    'integracao_metro',
    'bloqueio_auto_sistema',
    'target_fraude'
]

for col in binary_cols:
    df[col] = normalize_binary(df[col]).astype('Int64')

checkpoint("Variáveis binárias normalizadas para 0/1")

✔ Coluna 'bloqueio_automatico_sistema' renomeada para 'bloqueio_auto_sistema'
✔ Variáveis binárias normalizadas para 0/1


In [27]:
display(df[binary_cols].dtypes)
display(df[binary_cols].value_counts(dropna=False))


,0
feriado,Int64
integracao_metro,Int64
bloqueio_auto_sistema,Int64
target_fraude,Int64


feriado  integracao_metro  bloqueio_auto_sistema  target_fraude
0        0                 0                      0                12874
         1                 0                      0                12831
                           1                      1                 1421
         0                 1                      1                 1380
1        0                 0                      0                  687
         1                 0                      0                  673
                           1                      1                   73
         0                 1                      1                   61
Name: count, dtype: int64

<small>***Comentários Letícia:** As variáveis binárias foram normalizadas para o formato 0/1 e tiveram sua nomenclatura alinhada ao dicionário de dados. A validação posterior confirmou a ausência de valores inconsistentes, assegurando padronização semântica e compatibilidade com as etapas subsequentes de tratamento e modelagem.*</small>

### 4.3 Coerção de numéricas

In [29]:
numeric_cols = [
    'id_transacao',
    'id_cartao',
    'nu_recargas_acumulado',
    'valor_transacao',
    'id_dispositivo',
    'id_motorista',
    'linha_onibus',
    'temp_externa',
    'id_loja_ultima_recarga',
    'limite_diario_uso',
    'idade_usuario'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

checkpoint("Coerção explícita aplicada às colunas numéricas")


✔ Coerção explícita aplicada às colunas numéricas


In [30]:
int_cols = [
    'id_transacao',
    'id_cartao',
    'nu_recargas_acumulado',
    'id_dispositivo',
    'id_motorista',
    'linha_onibus',
    'id_loja_ultima_recarga',
    'limite_diario_uso',
    'idade_usuario'
]

float_cols = [
    'valor_transacao',
    'temp_externa'
]

for col in int_cols:
    df[col] = df[col].astype('Int64')

for col in float_cols:
    df[col] = df[col].astype(float)

checkpoint("Tipos numéricos padronizados (inteiros e contínuos)")


✔ Tipos numéricos padronizados (inteiros e contínuos)


In [33]:
display(df[int_cols + float_cols].dtypes)

display(
    df[int_cols + float_cols]
    .isna()
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

checkpoint("Item 4.3 concluído: variáveis numéricas coercidas e padronizadas")

,0
id_transacao,Int64
id_cartao,Int64
nu_recargas_acumulado,Int64
id_dispositivo,Int64
id_motorista,Int64
linha_onibus,Int64
id_loja_ultima_recarga,Int64
limite_diario_uso,Int64
idade_usuario,Int64
valor_transacao,float64


,0
id_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
id_dispositivo,0.0000
id_motorista,0.0000
linha_onibus,0.0000
id_loja_ultima_recarga,0.0000
limite_diario_uso,0.0000
idade_usuario,0.0000
valor_transacao,0.0000


✔ Item 4.3 concluído: variáveis numéricas coercidas e padronizadas


<small>***Comentários Letícia:** As variáveis numéricas foram explicitamente coercidas e padronizadas quanto ao tipo, garantindo separação adequada entre identificadores, contadores e variáveis contínuas. A validação posterior confirmou a ausência de valores inválidos ou ausentes introduzidos nesta etapa, indicando consistência numérica do conjunto de dados e preparando a base para as próximas fases de tratamento e engenharia de features.*</small>

### 4.4 Padronização de categóricas

In [34]:
categorical_cols = [
    'tipo_cartao',
    'status_cartao',
    'sentido_viagem',
    'clima_dia',
    'tipo_pagamento_recarga',
    'cidade',
    'sexo_usuario'
]

for col in categorical_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({'nan': pd.NA})
    )

checkpoint("Padronização básica aplicada às variáveis categóricas (lowercase + trim)")

✔ Padronização básica aplicada às variáveis categóricas (lowercase + trim)


In [35]:
for col in categorical_cols:
    df[col] = df[col].astype('category')

checkpoint("Variáveis categóricas convertidas para dtype 'category'")


✔ Variáveis categóricas convertidas para dtype 'category'


In [37]:
display(df[categorical_cols].dtypes)

# Cardinalidade por coluna
for col in categorical_cols:
    print(f"\n{col} - valores únicos:")
    print(df[col].value_counts(dropna=False).head(10))

checkpoint("Item 4.4 concluído: variáveis categóricas padronizadas")

,0
tipo_cartao,category
status_cartao,category
sentido_viagem,category
clima_dia,category
tipo_pagamento_recarga,category
cidade,category
sexo_usuario,category



tipo_cartao - valores únicos:
tipo_cartao
comum              6035
funcionario        6012
idoso              6006
estudante          5982
vale-transporte    5965
Name: count, dtype: int64

status_cartao - valores únicos:
status_cartao
ativo        28216
suspenso       920
bloqueado      864
Name: count, dtype: int64

sentido_viagem - valores únicos:
sentido_viagem
ida      15065
volta    14935
Name: count, dtype: int64

clima_dia - valores únicos:
clima_dia
chuva         10063
ensolarado    10003
nublado        9934
Name: count, dtype: int64

tipo_pagamento_recarga - valores únicos:
tipo_pagamento_recarga
pix               10149
cartao_credito     9957
dinheiro           9894
Name: count, dtype: int64

cidade - valores únicos:
cidade
são paulo    30000
Name: count, dtype: int64

sexo_usuario - valores únicos:
sexo_usuario
f    10019
m    10006
n     9975
Name: count, dtype: int64
✔ Item 4.4 concluído: variáveis categóricas padronizadas


<small>***Comentários Letícia:** As variáveis categóricas foram padronizadas quanto à grafia e convertidas para o tipo category, assegurando consistência semântica e reduzindo ruído nos dados. A validação posterior indicou categorias bem definidas e distribuição equilibrada na maioria das variáveis, com destaque para a coluna cidade, que apresentou ausência de variabilidade e deverá ser reavaliada quanto à sua utilidade nas próximas etapas do pipeline.*</small>



### 4.5 Checkpoint de tipos finais

In [39]:
checkpoint("Visão consolidada do schema após padronização")
display(df.dtypes)


✔ Visão consolidada do schema após padronização


,0
id_transacao,Int64
id_cartao,Int64
ts_transacao,datetime64[ns]
dt_emissao_cartao,datetime64[ns]
nu_recargas_acumulado,Int64
valor_transacao,float64
tipo_cartao,category
status_cartao,category
id_dispositivo,Int64
id_motorista,Int64


In [40]:
checkpoint("Percentual de valores ausentes por coluna (top 15)")
display(
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
      .head(15)
)


✔ Percentual de valores ausentes por coluna (top 15)


,0
diff_tempo_segundos,31.6100
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000
id_motorista,0.0000


In [43]:
expected_schema = [
    "id_transacao","id_cartao","ts_transacao","dt_emissao_cartao",
    "nu_recargas_acumulado","valor_transacao","tipo_cartao","status_cartao",
    "id_dispositivo","id_motorista","linha_onibus","sentido_viagem",
    "clima_dia","temp_externa","id_loja_ultima_recarga","tipo_pagamento_recarga",
    "latitude","longitude","cidade","feriado","integracao_metro",
    "limite_diario_uso","idade_usuario","sexo_usuario",
    "bloqueio_auto_sistema","target_fraude"
]

extra_cols = [c for c in df.columns if c not in expected_schema]
missing_cols = [c for c in expected_schema if c not in df.columns]

if extra_cols:
    print("⚠️ Colunas fora do schema esperado:", extra_cols)
else:
    checkpoint("Nenhuma coluna extra fora do schema esperado.")

if missing_cols:
    print("⚠️ Colunas esperadas ausentes:", missing_cols)
else:
    checkpoint("Nenhuma coluna esperada ausente.")

checkpoint("Etapa 4 concluída: schema padronizado e validado")

⚠️ Colunas fora do schema esperado: ['diff_tempo_segundos', 'tempo_vida_cartao_dias']
✔ Nenhuma coluna esperada ausente.
✔ Etapa 4 concluída: schema padronizado e validado


<small>***Comentários Letícia:** Ao final desta etapa, o schema do dataset foi padronizado e validado com sucesso, contemplando a conversão adequada de tipos temporais, numéricos, categóricos e binários. A validação final confirmou o alinhamento com o dicionário de dados, bem como a presença de colunas derivadas não previstas no conjunto bruto, que serão tratadas na próxima etapa. Com isso, a base encontra-se estruturalmente consistente e pronta para o tratamento de qualidade e engenharia de features.*</small>

## 5. Tratamento de inconsistências e qualidade dos dados

### 5.1 Remoção de duplicatas completas

In [47]:
# Remoção das colunas

cols_to_drop = [
    'diff_tempo_segundos',
    'tempo_vida_cartao_dias'
]

df = df.drop(columns=cols_to_drop, errors='ignore')

checkpoint(f"Colunas removidas do dataset base: {cols_to_drop}")

✔ Colunas removidas do dataset base: ['diff_tempo_segundos', 'tempo_vida_cartao_dias']


In [46]:
checkpoint("Validação do schema após remoção de colunas indevidas")

display(df.columns)

checkpoint("Item 5.1 concluído: colunas fora do schema removidas")

✔ Validação do schema após remoção de colunas indevidas


Index(['id_transacao', 'id_cartao', 'ts_transacao', 'dt_emissao_cartao',
       'nu_recargas_acumulado', 'valor_transacao', 'tipo_cartao',
       'status_cartao', 'id_dispositivo', 'id_motorista', 'linha_onibus',
       'sentido_viagem', 'clima_dia', 'temp_externa', 'id_loja_ultima_recarga',
       'tipo_pagamento_recarga', 'latitude', 'longitude', 'cidade', 'feriado',
       'integracao_metro', 'limite_diario_uso', 'idade_usuario',
       'sexo_usuario', 'target_fraude', 'bloqueio_auto_sistema'],
      dtype='object')

✔ Item 5.1 concluído: colunas fora do schema removidas


<small>***Comentários Letícia:** Neste passo, removi do dataset base as colunas derivadas não previstas no dicionário de dados, previamente identificadas como inconsistências estruturais. Com isso, o conjunto de dados passa a refletir fielmente o dado bruto padronizado, preservando a integridade do schema e evitando riscos de vazamento de informação nas etapas subsequentes.*</small>

### 5.2 Regra para id_transacao duplicado (decisão e aplicação)

In [48]:
# Verificação de duplicidade de linhas completas
dup_linhas = df.duplicated().sum()

checkpoint(f"Linhas duplicadas completas identificadas: {dup_linhas}")

✔ Linhas duplicadas completas identificadas: 0


In [49]:
# Verificação de duplicidade da chave primária (id_transacao)
dup_transacao = df['id_transacao'].duplicated().sum()

checkpoint(f"Duplicidades na chave id_transacao: {dup_transacao}")

✔ Duplicidades na chave id_transacao: 0


In [50]:
# Verificação de cardinalidade das chaves
checkpoint("Validação de cardinalidade das principais chaves")

print("Total de registros:", len(df))
print("Transações únicas:", df['id_transacao'].nunique())
print("Cartões únicos:", df['id_cartao'].nunique())

✔ Validação de cardinalidade das principais chaves
Total de registros: 30000
Transações únicas: 30000
Cartões únicos: 9483


In [52]:
# Validação cruzada simples (consistência transação × cartão)
inconsistencias_cartao = (
    df.groupby('id_transacao')['id_cartao']
      .nunique()
      .gt(1)
      .sum()
)

checkpoint(f"Transações associadas a mais de um cartão: {inconsistencias_cartao}")
checkpoint("Item 5.2 concluído: duplicidades e chaves validadas")

✔ Transações associadas a mais de um cartão: 0
✔ Item 5.2 concluído: duplicidades e chaves validadas


<small>***Comentários Letícia:** A verificação de duplicidades e integridade das chaves confirmou que o conjunto de dados não apresenta registros duplicados nem inconsistências na identificação das transações e dos cartões. Essa validação assegura a confiabilidade da base para operações de agregação, análise temporal e engenharia de features nas etapas subsequentes.*</small>



### 5.3 Tratamento de valores inválidos

In [53]:
# Validação de faixas plausíveis (sanity checks de domínio)

checkpoint("Validação de faixas plausíveis – sanity checks")

sanity_checks = {
    'idade_usuario': (0, 120),
    'valor_transacao': (0, None),
    'temp_externa': (-20, 60),
    'latitude': (-90, 90),
    'longitude': (-180, 180)
}

for col, (min_val, max_val) in sanity_checks.items():
    if max_val is not None:
        invalid = df[(df[col] < min_val) | (df[col] > max_val)][col].count()
    else:
        invalid = df[df[col] < min_val][col].count()

    print(f"{col}: valores inválidos identificados = {invalid}")

✔ Validação de faixas plausíveis – sanity checks
idade_usuario: valores inválidos identificados = 57
valor_transacao: valores inválidos identificados = 0
temp_externa: valores inválidos identificados = 0
latitude: valores inválidos identificados = 0
longitude: valores inválidos identificados = 0


In [54]:
# Tratamento conservador de valores inválidos

for col, (min_val, max_val) in sanity_checks.items():
    if max_val is not None:
        df.loc[(df[col] < min_val) | (df[col] > max_val), col] = pd.NA
    else:
        df.loc[df[col] < min_val, col] = pd.NA

checkpoint("Valores fora de faixa tratados como ausentes (NaN)")

✔ Valores fora de faixa tratados como ausentes (NaN)


In [56]:
# Validação pós-tratamento

checkpoint("Percentual de valores ausentes após tratamento de inconsistências")

display(
    df[list(sanity_checks.keys())]
      .isna()
      .mean()
      .mul(100)
      .round(2)
)

checkpoint("Item 5.3 concluído: valores inválidos tratados de forma conservadora")

✔ Percentual de valores ausentes após tratamento de inconsistências


,0
idade_usuario,0.1900
valor_transacao,0.0000
temp_externa,0.0000
latitude,0.0000
longitude,0.0000


✔ Item 5.3 concluído: valores inválidos tratados de forma conservadora


<small>***Comentários Letícia:** A validação de faixas plausíveis indicou inconsistências pontuais apenas na variável idade do usuário, representando uma fração muito pequena do conjunto de dados. Esses valores foram tratados de forma conservadora, sendo convertidos em ausentes, sem exclusão de registros ou imputações arbitrárias. O impacto do saneamento foi mínimo, preservando a qualidade e a integridade do dataset para as próximas etapas.*</small>

### 5.4 Análise de nulos

In [57]:
# Identificação de variáveis constantes

checkpoint("Identificação de variáveis com variância zero")

constantes = [
    col for col in df.columns
    if df[col].nunique(dropna=False) <= 1
]

print("Variáveis constantes identificadas:", constantes)

✔ Identificação de variáveis com variância zero
Variáveis constantes identificadas: ['cidade', 'limite_diario_uso']


In [58]:
# Remoção das variáveis sem variabilidade

df = df.drop(columns=constantes, errors='ignore')

checkpoint(f"Variáveis sem poder informativo removidas: {constantes}")

✔ Variáveis sem poder informativo removidas: ['cidade', 'limite_diario_uso']


In [59]:
# Validação pós-remoção

checkpoint("Validação do schema após remoção de variáveis constantes")
display(df.columns)

checkpoint("Item 5.4 concluído: variáveis sem variância removidas")

✔ Validação do schema após remoção de variáveis constantes


Index(['id_transacao', 'id_cartao', 'ts_transacao', 'dt_emissao_cartao',
       'nu_recargas_acumulado', 'valor_transacao', 'tipo_cartao',
       'status_cartao', 'id_dispositivo', 'id_motorista', 'linha_onibus',
       'sentido_viagem', 'clima_dia', 'temp_externa', 'id_loja_ultima_recarga',
       'tipo_pagamento_recarga', 'latitude', 'longitude', 'feriado',
       'integracao_metro', 'idade_usuario', 'sexo_usuario', 'target_fraude',
       'bloqueio_auto_sistema'],
      dtype='object')

✔ Item 5.4 concluído: variáveis sem variância removidas


<small>***Comentários Letícia:** A análise de variabilidade identificou colunas que apresentavam valor constante em todo o conjunto de dados, não oferecendo qualquer poder discriminativo para a identificação de fraude. As variáveis cidade e limite_diario_uso foram removidas, reduzindo ruído e complexidade do dataset, sem impacto negativo na informação relevante para as próximas etapas do pipeline.*</small>

### 5.5 Tratamento mínimo de nulos críticos


In [60]:
# Percentual final de valores ausentes por coluna

checkpoint("Percentual final de valores ausentes por coluna")

display(
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
)

✔ Percentual final de valores ausentes por coluna


,0
idade_usuario,0.1900
id_transacao,0.0000
ts_transacao,0.0000
id_cartao,0.0000
nu_recargas_acumulado,0.0000
valor_transacao,0.0000
tipo_cartao,0.0000
dt_emissao_cartao,0.0000
status_cartao,0.0000
id_dispositivo,0.0000


In [61]:
# Visão resumida das colunas com valores ausentes

cols_com_na = df.columns[df.isna().any()].tolist()

print("Colunas com valores ausentes:", cols_com_na)

Colunas com valores ausentes: ['idade_usuario']


In [63]:
# Avaliação do impacto dos valores ausentes

checkpoint("Impacto dos valores ausentes no dataset")

total_linhas = len(df)
linhas_com_na = df.isna().any(axis=1).sum()

print(f"Total de registros: {total_linhas}")
print(f"Registros com pelo menos um valor ausente: {linhas_com_na}")
print(f"Percentual de registros afetados: {linhas_com_na / total_linhas * 100:.2f}%")

checkpoint("Item 5.5 concluído: valores ausentes analisados e consolidados")

✔ Impacto dos valores ausentes no dataset
Total de registros: 30000
Registros com pelo menos um valor ausente: 57
Percentual de registros afetados: 0.19%
✔ Item 5.5 concluído: valores ausentes analisados e consolidados


<small>***Comentários Letícia:** A análise final de valores ausentes indicou impacto mínimo no conjunto de dados, restrito a uma pequena parcela de registros na variável idade do usuário. Considerando a baixa representatividade desses casos e sua origem conhecida, optei por não realizar imputações ou exclusões nesta etapa, mantendo o dataset íntegro e preservando decisões para fases posteriores, onde o contexto analítico poderá ser melhor avaliado.*</small>

### 5.6 Checkpoint de qualidade pós-tratamento

In [64]:
checkpoint("Checkpoint final de qualidade pós-tratamento")

# Visão geral do dataset
print("Shape final do dataset:", df.shape)
display(df.head())

# Métricas de integridade
print("\nResumo de integridade:")
print("Registros totais:", len(df))
print("Transações únicas:", df['id_transacao'].nunique())
print("Cartões únicos:", df['id_cartao'].nunique())

# Valores ausentes
perc_linhas_na = df.isna().any(axis=1).mean() * 100
print(f"\nPercentual de registros com ao menos um valor ausente: {perc_linhas_na:.2f}%")
print("Colunas com valores ausentes:", df.columns[df.isna().any()].tolist())

# Duplicidades
print("\nChecagem de duplicidades:")
print("Linhas duplicadas completas:", df.duplicated().sum())
print("Duplicatas em id_transacao:", df['id_transacao'].duplicated().sum())

checkpoint("Etapa 5 concluída: qualidade dos dados validada e consolidada")

✔ Checkpoint final de qualidade pós-tratamento
Shape final do dataset: (30000, 24)


,id_transacao,id_cartao,ts_transacao,dt_emissao_cartao,nu_recargas_acumulado,valor_transacao,tipo_cartao,status_cartao,id_dispositivo,id_motorista,linha_onibus,sentido_viagem,clima_dia,temp_externa,id_loja_ultima_recarga,tipo_pagamento_recarga,latitude,longitude,feriado,integracao_metro,idade_usuario,sexo_usuario,target_fraude,bloqueio_auto_sistema
0,160,18901,2026-01-21 20:23:29,2022-10-17,27,4.5000,funcionario,ativo,560,1155,88,volta,chuva,24.8087,15,cartao_credito,-23.5194,-46.6360,0,0,26,n,0,0
1,103,18392,2026-01-29 03:13:36,2023-06-30,142,4.5000,idoso,ativo,597,1184,99,volta,chuva,26.5893,44,dinheiro,-23.5968,-46.6113,0,1,39,f,0,0
2,22320,17050,2026-01-04 16:05:52,2020-10-29,51,4.5000,comum,ativo,555,1118,7,volta,chuva,37.0170,13,pix,-23.5743,-46.6591,0,0,45,f,0,0
3,12932,11516,2026-01-15 01:29:07,2023-09-23,34,9.0000,idoso,ativo,500,1152,129,volta,chuva,29.3219,26,pix,-23.5572,-46.6581,0,1,20,n,0,0
4,21,16396,2026-01-19 00:20:30,2023-01-10,78,4.5000,vale-transporte,ativo,563,1078,111,ida,chuva,24.1620,12,cartao_credito,-23.5435,-46.6858,0,0,45,m,0,0



Resumo de integridade:
Registros totais: 30000
Transações únicas: 30000
Cartões únicos: 9483

Percentual de registros com ao menos um valor ausente: 0.19%
Colunas com valores ausentes: ['idade_usuario']

Checagem de duplicidades:
Linhas duplicadas completas: 0
Duplicatas em id_transacao: 0
✔ Etapa 5 concluída: qualidade dos dados validada e consolidada


<small>***Comentários Letícia:** Ao final desta etapa, o conjunto de dados apresentou elevada qualidade estrutural e semântica, com integridade total das chaves, ausência de duplicidades e impacto mínimo de valores ausentes. As inconsistências identificadas foram tratadas de forma conservadora, preservando a informação relevante e evitando decisões arbitrárias. Com isso, o dataset encontra-se consolidado, limpo e confiável, estando apto para a etapa de engenharia de features e posterior modelagem preditiva.*</small>

## 6. Engenharia de features

### 6.1 Ordenação base

<small>***Comentários Letícia:** ttt*</small>

### 6.2 Features temporais básicas

<small>***Comentários Letícia:** ttt*</small>

### 6.3 Features de sequência

<small>***Comentários Letícia:** ttt*</small>

### 6.4 Frequência diária

<small>***Comentários Letícia:** ttt*</small>

### 6.5 Frequência 24h

<small>***Comentários Letícia:** ttt*</small>

### 6.6 Flags de intensidade

<small>***Comentários Letícia:** ttt*</small>

### 6.7 Consistência operacional



<small>***Comentários Letícia:** ttt*</small>

### 6.8 Diversidade diária

<small>***Comentários Letícia:** ttt*</small>

### 6.9 Features de contexto robustas

<small>***Comentários Letícia:** ttt*</small>

### 6.10 Faixas de valor

<small>***Comentários Letícia:** ttt*</small>

### 6.11 Agregações por cartão

<small>***Comentários Letícia:** ttt*</small>

### 6.12 Checkpoint geral das features

<small>***Comentários Letícia:** ttt*</small>

## 7. Seleção final de colunas e preparação do dataset processado

### 7.1 Definição de colunas de rastreio, target e features

<small>***Comentários Letícia:** ttt*</small>

### 7.2 Montagem do dataset final

<small>***Comentários Letícia:** ttt*</small>

### 7.3 Validação de dtypes finais e cardinalidade de categóricas

<small>***Comentários Letícia:** ttt*</small>

### 7.4 Auditoria final de nulos

<small>***Comentários Letícia:** ttt*</small>

### 7.5 Checkpoint do dataset final

<small>***Comentários Letícia:** ttt*</small>

## 8. Exportação dos dados processados

### 8.1 Export do dataset processado

<small>***Comentários Letícia:** ttt*</small>

### 8.2 Export do schema de colunas (txt/json)

<small>***Comentários Letícia:** ttt*</small>

### 8.3 Export de dtypes e metadados de processamento

<small>***Comentários Letícia:** ttt*</small>

### 8.4 Checkpoint de exportação

<small>***Comentários Letícia:** ttt*</small>

## 9. Insights Finais